In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import string
import numpy as np
import pandas as pd
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
def google_search(x,n):
    driver = webdriver.Chrome('chromedriver')
    driver.get(f'https://www.google.com/search?q={x}')
    next_page_times = n
    http=[]
    for _page in range(next_page_times):
    
        soup = BeautifulSoup(driver.page_source,'html.parser')
        items=soup.find_all('div', attrs={'class':'yuRUbf'})

        for i in items:
            # 網址
            h=i.find('a')['href']
            if 'cdc.gov.tw' in h :
                pass
            else:
                http.append(h)
        # Wait
        time.sleep(random.uniform(1, 5))

        # Turn to the next page
        try:
            driver.find_element_by_link_text('下一頁').click()
        except:
            print('Search Early Stopping.')
            break
    driver.close()
    return http

In [3]:
def count_word(x):
    chinese=[]
    english=[]
    number=[]
    en=''
    num=''
    for i in list(x):

        if i in string.ascii_letters:
            en+=i
        elif i.isdigit():
            num+=i
        else:
            if en !='':
                english.append(en)
            if num!='':
                number.append(num)
            en=''
            num=''
            if i !=' ':
                chinese.append(i)
    words=len(english)+len(chinese)+len(number)
    return words

In [4]:
def chinatime(x):
    china=BeautifulSoup(x.text, 'html.parser')
    #標題
    try:
        titles=china.find('h1',attrs={'class':'article-title'}).text
    except:
        titles='Error'
    #日期
    try:
        date=china.find('meta', property="article:published_time")['content'][0:10]
    except :
        date='Error'
    #記者
    try:
        author=china.find('div',attrs={'class':'author'}).text
        author=author.replace('\n','')
        author=author.replace(' ','')
    except:
        author='Error'
    #類別
    try:
        sention=china.find('meta',attrs={'property':'article:section'})['content']
    except:
        sention='Error'
    #內文
    try:
        content=china.find('div',attrs={'class':'article-body'}).text
        content=content.replace('\n','')
        k=content.rfind('。')
        content=content[:k+1]
    except:
        content='Error'
    #字數
    try:
        words=count_word(content)
    except:
        words='Error'
    try:
        #圖片數
        n_p=len(china.find_all('div',attrs={'class':'photo-container'}))
        #影片數
        v_p=len(china.find_all('div',attrs={'class':'article-body','class':'video-container'}))
        ##加起來
        if v_p !=0:
            all_p=f'{n_p}（+{v_p}影片）'
        else:
            all_p=n_p
    except:
        all_p='Error'
    if '疫苗' in content :
        return ['','',author,'',sention,date,titles,content,'','','','','',words,'',all_p,'']
    elif content=='錯誤' :
        return ['','',author,'',sention,date,titles,content,'','','','','',words,'',all_p,'']
    if words==1:
        return ['','',author,'',sention,date,titles,content,'','','','','',words,'',all_p,'']
    else:
        return['','','','','','','','','','','','','','','','','']

In [5]:
def web_crawl(x,n): 
    http=google_search(x,n)
    china_time=[]
    for i in range(0,len(http)):
        china_time.append(requests.get(http[i]))
        time.sleep(random.uniform(1, 5))
    china_time_result=list(map(chinatime,china_time))
    chinatime_df=pd.DataFrame(china_time_result)
    chinatime_df=chinatime_df.rename(index=str,columns={0:'',1:'',2:'作者',3:'',4:'記者路線',5:'日期',6:'標題',7:'內文',8:'',9:'',10:'',11:'',12:'',13:'字數',14:'',15:"附圖（幀）",16:''})
    chinatime_df['網址']=http
    chinatime_df=chinatime_df[chinatime_df.標題!='']
    return chinatime_df

In [6]:
df=web_crawl('疫苗 after:2021-07-08 before:2021-07-10 site:chinatimes.com',21)

Search Early Stopping.


In [7]:
df

,,,作者,,記者路線,日期,標題,內文,,,,,,字數,,附圖（幀）,,網址
0,,,吳美觀、中時、吳靜君,,財經,2021-07-10,高端員工遭爆搶打公費疫苗？公司回應了,針對外界傳聞，高端的員工已施打新冠肺炎疫苗之事，高端今日發重訊說明，至今只有一名員工因懷孕以...,,,,,,291,,1,,https://www.chinatimes.com/realtimenews/202107...
1,,,陳慰慈,,生活,2021-07-09,新北10、11日開放67歲以上長者預約打疫苗,新北市10日、11日開放不限戶籍67歲以上長者、57歲以上原住民，及不限戶籍孕婦、已列冊的第...,,,,,,238,,1,,https://www.chinatimes.com/realtimenews/202107...
2,,,趙婉淳、林周義、林良齊,,政治,2021-07-09,陳時中認了高端、聯亞人員打國外疫苗：這些人染疫我們也睡不著覺,憂高端、聯亞人員染疫會睡不著？ 陳時中喊冤：指封緘檢驗人員訂閱影音：中時新聞網國內疫苗短缺，...,,,,,,1173,,1（+1影片）,,https://www.chinatimes.com/realtimenews/202107...
3,,,何立雯,,生活,2021-07-10,新冠疫苗分葷素、BNT含蛋？前台大醫神解真相,近日網上謠傳疫苗有葷素之分，稱「莫德納疫苗是純素、輝瑞疫苗有蛋」。對此，台大前感染科醫師林氏...,,,,,,733,,1,,https://www.chinatimes.com/realtimenews/202107...
4,,,李俊毅,,政治,2021-07-09,逾3百人打疫苗死亡 藍委脫口一句怒嗆蔡政府,國內外購、美日外援新冠疫苗逐步進入台灣，近期開始陸續施打，因目前疫苗施打率不佳，以及有不少長...,,,,,,627,,2,,https://www.chinatimes.com/realtimenews/202107...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,,,陳弘美,,Error,2021-07-09,疫苗要不要打第3針 陳文茜：美國FDA宣布「答案」,新冠肺炎疫情再起，需要不要打第3針疫苗加強抗體成為外界關注焦點。（圖為輝瑞疫苗，達志影像／s...,,,,,,621,,1（+1影片）,,https://www.chinatimes.com/amp/realtimenews/20...
199,,,張妍溱,,Error,2021-07-09,台中疫苗打氣旺 3天逾9.7萬人接種莫德納,台中疫苗打氣旺 3天逾9.7萬人接種莫德納。（示意圖／達志影像）台中市9日本土、境外、社區快...,,,,,,426,,2,,https://www.chinatimes.com/amp/realtimenews/20...
200,,,游念育,,生活,2021-07-10,議員爆北市8名幼教員僅7人可打疫苗 北市教育局回應了,台北市長柯文哲日前宣布放寬托育、身障日照機構收托，但員工必須先施打疫苗，北市議員簡舒培卻在臉...,,,,,,604,,1,,https://www.chinatimes.com/realtimenews/202107...
202,,,郭家崴,,時事,2021-07-08,台北101五句話點燈 感謝日本提供113萬劑疫苗,為感謝日本再次提供台灣113萬劑疫苗，成為台灣對抗疫情最堅實的力量，也展現台日深厚友誼，台北...,,,,,,117,,1,,https://www.chinatimes.com/realtimenews/202107...


In [8]:
df2=df.drop_duplicates()

In [9]:
df2

,,,作者,,記者路線,日期,標題,內文,,,,,,字數,,附圖（幀）,,網址
0,,,吳美觀、中時、吳靜君,,財經,2021-07-10,高端員工遭爆搶打公費疫苗？公司回應了,針對外界傳聞，高端的員工已施打新冠肺炎疫苗之事，高端今日發重訊說明，至今只有一名員工因懷孕以...,,,,,,291,,1,,https://www.chinatimes.com/realtimenews/202107...
1,,,陳慰慈,,生活,2021-07-09,新北10、11日開放67歲以上長者預約打疫苗,新北市10日、11日開放不限戶籍67歲以上長者、57歲以上原住民，及不限戶籍孕婦、已列冊的第...,,,,,,238,,1,,https://www.chinatimes.com/realtimenews/202107...
2,,,趙婉淳、林周義、林良齊,,政治,2021-07-09,陳時中認了高端、聯亞人員打國外疫苗：這些人染疫我們也睡不著覺,憂高端、聯亞人員染疫會睡不著？ 陳時中喊冤：指封緘檢驗人員訂閱影音：中時新聞網國內疫苗短缺，...,,,,,,1173,,1（+1影片）,,https://www.chinatimes.com/realtimenews/202107...
3,,,何立雯,,生活,2021-07-10,新冠疫苗分葷素、BNT含蛋？前台大醫神解真相,近日網上謠傳疫苗有葷素之分，稱「莫德納疫苗是純素、輝瑞疫苗有蛋」。對此，台大前感染科醫師林氏...,,,,,,733,,1,,https://www.chinatimes.com/realtimenews/202107...
4,,,李俊毅,,政治,2021-07-09,逾3百人打疫苗死亡 藍委脫口一句怒嗆蔡政府,國內外購、美日外援新冠疫苗逐步進入台灣，近期開始陸續施打，因目前疫苗施打率不佳，以及有不少長...,,,,,,627,,2,,https://www.chinatimes.com/realtimenews/202107...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,,,陳弘美,,Error,2021-07-09,疫苗要不要打第3針 陳文茜：美國FDA宣布「答案」,新冠肺炎疫情再起，需要不要打第3針疫苗加強抗體成為外界關注焦點。（圖為輝瑞疫苗，達志影像／s...,,,,,,621,,1（+1影片）,,https://www.chinatimes.com/amp/realtimenews/20...
199,,,張妍溱,,Error,2021-07-09,台中疫苗打氣旺 3天逾9.7萬人接種莫德納,台中疫苗打氣旺 3天逾9.7萬人接種莫德納。（示意圖／達志影像）台中市9日本土、境外、社區快...,,,,,,426,,2,,https://www.chinatimes.com/amp/realtimenews/20...
200,,,游念育,,生活,2021-07-10,議員爆北市8名幼教員僅7人可打疫苗 北市教育局回應了,台北市長柯文哲日前宣布放寬托育、身障日照機構收托，但員工必須先施打疫苗，北市議員簡舒培卻在臉...,,,,,,604,,1,,https://www.chinatimes.com/realtimenews/202107...
202,,,郭家崴,,時事,2021-07-08,台北101五句話點燈 感謝日本提供113萬劑疫苗,為感謝日本再次提供台灣113萬劑疫苗，成為台灣對抗疫情最堅實的力量，也展現台日深厚友誼，台北...,,,,,,117,,1,,https://www.chinatimes.com/realtimenews/202107...


In [10]:
df2.to_excel('中時0708-0710.xlsx')

False

In [133]:
#news[0].to_excel('0101到0111.xlsx')

In [ ]:
new[1]

In [3]:
r=[]

In [4]:
for i in range(0,len(web_list)):
    r1 = requests.get(web_list[i])
    r.append(r1)
    time.sleep(random.uniform(1, 5))

In [5]:
import pandas as pd
df=pd.DataFrame()

In [6]:
title=[]
http=[]

In [7]:
for i in range(0,len(r)):
    if r[0].status_code == requests.codes.ok:
      # 以 BeautifulSoup 解析 HTML 原始碼
        soup = BeautifulSoup(r[0].text, 'html.parser')

      #觀察 HTML 原始碼
      # print(soup.prettify())

      # 以 CSS 的選擇器來抓取 Google 的搜尋結果
        items = soup.select('a[href^="/url"]')

        items= items[:-2]
        for i in items:
        # 標題
            title.append(i.text)
        # 網址
            http.append(i.get('href')[7:].split('&')[0])

In [8]:
dic={"標題":title,"網址":http}
df=pd.DataFrame(dic)

In [110]:
#df

In [11]:
china_time=[]
for i in range(0,len(http)):
    china_time.append(requests.get(http[i]))
    time.sleep(random.uniform(1, 5))

In [12]:
def count_word(x):
    chinese=[]
    english=[]
    number=[]
    en=''
    num=''
    for i in list(x):

        if i in string.ascii_letters:
            en+=i
        elif i.isdigit():
            num+=i
        else:
            if en !='':
                english.append(en)
            if num!='':
                number.append(num)
            en=''
            num=''
            if i !=' ':
                chinese.append(i)
    words=len(english)+len(chinese)+len(number)
    return words

In [92]:
def chinatime(x):
    china=BeautifulSoup(x.text, 'html.parser')
    #標題
    title=china.find('h1', attrs={'class':'article-title'}).text
    #日期
    date=china.find('meta', property="article:published_time")['content'][0:10]
    #記者
    author=china.find('div',attrs={'class':'author'}).text
    author=author.replace('\n','')
    author=author.replace(' ','')
    #類別
    sention=china.find('meta',attrs={'property':'article:section'})['content']
    #內文
    content=china.find('div',attrs={'class':'article-body'}).text
    content=content.replace('\n','')
    k=content.rfind('。')
    content=content[:k+1]
    #字數
    words=count_word(content)
    #圖片數
    n_p=len(china.find_all('div',attrs={'class':'photo-container'}))
    #影片數
    v_p=len(china.find_all('div',attrs={'class':'article-body','class':'video-container'}))
    ##加起來
    if v_p !=0:
        all_p=f'{n_p}+{v_p}影片'
    else:
        all_p=n_p
    if '疫苗' in content :
        return [title,content,words,sention,date,author,all_p]
    else:
        return['','','','','','','']

In [93]:
china_time_result=list(map(chinatime,china_time))

In [94]:
china_time_result[1]

['輝瑞疫苗 獲世衛第一個認證施打',
 '世界衛生組織（WHO）於元旦前夕宣布，將其第一個新冠疫苗緊急使用認證授予美國輝瑞藥廠和德國BioNTech公司共同研發的疫苗，並特別提醒這款疫苗需要以超低溫方式儲存，這對配送工作形成相當挑戰。WHO召集來自世界各地的專家與WHO團隊一起全面審核了該疫苗的安全性、有效性和質量數據，認為這款疫苗符合WHO制定的安全性和有效性標準，使用該疫苗因應新冠疫情帶來的好處足以抵消其潛在風險。WHO負責藥品和衛生產品的助理祕書長西蒙女士表示，這是確保全球獲得新冠疫苗非常重要的一步。但她強調，全球需要做出更大努力，以實現有充足的疫苗供應來滿足世界各地重點人群的需求。WHO免疫策略諮詢小組將於1月5日召集會議，就去年9月公布的新冠疫苗接種人群總體優先次序建議，為該產品的使用制定具體政策和建議。WHO表示，輝瑞/BioNTech疫苗需要儲存在攝氏零下60至90度的低溫環境下，這對無法提供相應環境的地區形成極大挑戰，為此WHO正努力協助各國評估相關的執行計畫。另外，WHO也正式通報新冠病毒自出現以來4種變種病毒的主要變異情況，建議各國增加對新冠病毒的常規基因測序，以便更好地了解病毒傳播並監測其變種。根據通報，2020年1月底至2月初，病毒出現D614G突變，逐漸取代最初發現的病毒株，至2020年6月已成為全球主要傳播的病毒類型。2020年8至9月，丹麥發現一種與水貂相關的變種，被命名為Cluster 5，丹麥僅在9月發現12例人體感染此變種的病例，顯然未廣泛傳播。2020年12月14日，英國向WHO報告VOC 202012 /01新變種，具有更強傳播力，至12月30日，已有其他31個國家和地區發現此變種。12月18日，南非檢測到又一變種，命名為501Y.V2，南非以外已有4國發現該變種。',
 666,
 '生活新聞',
 '2021-01-02',
 '楊明暐',
 1]

In [95]:
chinatime_df=pd.DataFrame(china_time_result)
chinatime_df=chinatime_df.rename(index=str,columns={0:'標題',1:'內文',2:'字數',3:'分類',4:'日期',5:'記者/機構',6:"附圖（幀）"})

In [96]:
chinatime_df['網址']=http

In [100]:
chinatime_df

,標題,內文,字數,分類,日期,記者/機構,附圖（幀）,網址
0,印度將正式批准牛津疫苗 接種前全國先預演,印度武漢肺炎疫苗接種進入倒數階段，英國牛津大學與阿斯特捷利康（AstraZeneca）藥廠開...,1014,國際,2021-01-02,中央社,1,https://www.chinatimes.com/realtimenews/202101...
1,輝瑞疫苗 獲世衛第一個認證施打,世界衛生組織（WHO）於元旦前夕宣布，將其第一個新冠疫苗緊急使用認證授予美國輝瑞藥廠和德國B...,666,生活新聞,2021-01-02,楊明暐,1,https://www.chinatimes.com/newspapers/20210102...
2,日媒稱大陸製造新冠疫苗被帶入日本 中使館駁斥,中國駐日本大使館2日駁斥日媒報導「中國製造」疫苗流入日本傳聞。強調中方對個別日本媒體未經核實...,701,兩岸,2021-01-02,李文輝,1,https://www.chinatimes.com/realtimenews/202101...
3,以色列240人注射輝瑞疫苗後染疫 還有4死,以色列先前宣布，採用美國藥廠輝瑞與德國藥廠BioNTech合作研發的新冠疫苗，並展開全國接種...,481,國際,2021-01-02,施施,1,https://www.chinatimes.com/realtimenews/202101...
4,陸國產疫苗 全民免費施打,目前免費接種新冠疫苗的國家大陸首款國產新冠疫苗獲准上市，將提供全民免費施打，已450萬大陸民...,690,生活新聞,2021-01-01,藍孝威、諶悠文,2,https://www.chinatimes.com/newspapers/20210101...
5,不想掉隊 體育署得硬起來,全球體壇今年希望在新冠疫苗上路後，逐漸恢復原有秩序。台灣堪稱防疫模範生，但今年要「走出去」爭...,549,運動天地,2021-01-01,李弘斌、新聞透視,1,https://www.chinatimes.com/newspapers/20210101...
6,中時社論》比疫情還失控的陳時中,台灣出現英國變種病毒確診個案，才剛說過不必靠鎖國防疫的中央流行疫情指揮官陳時中，也不得不宣布...,1541,中時社論,2021-01-02,主筆室,1,https://www.chinatimes.com/opinion/20210102003...
7,疫情衝擊 藥廠調漲美國逾300款藥品價格,新冠疫情衝擊藥品需求，另一方面為反制美國政府祭出調降新藥價格的規範，輝瑞（Pfizer）、賽...,661,財經,2021-01-02,吳慧珍、綜合外電,0,https://www.chinatimes.com/realtimenews/202101...
8,,,,,,,,https://www.chinatimes.com/realtimenews/202101...
9,,,,,,,,https://www.chinatimes.com/realtimenews/202101...


In [104]:
chinatime_df.網址[7]

'https://www.chinatimes.com/realtimenews/20210102003792-260410'

In [107]:
chinatime_df.內文[7]

'新冠疫情衝擊藥品需求，另一方面為反制美國政府祭出調降新藥價格的規範，輝瑞（Pfizer）、賽諾菲（Sanofi）、葛蘭素史克（GSK）等各大藥廠打算自元旦起，調漲美國境內販售的300多種藥品價格。《路透》報導指出，美國民眾礙於疫情少上醫院看診，部分藥品的需求減少。此外美國處方藥價格之高在全球數一數二，讓處方藥降價早被川普列為競選政見，他上台後更是施政重點，但相關規範會壓縮製藥業利潤空間。全球大封鎖期間，病患求診和開處方藥暴減，輝瑞等大藥廠又挺身而出投入新冠疫苗研發，調漲藥品價格有助彌補藥廠流失的營收。據醫療保健研究機構3 Axis Advisors指出，各大藥廠的藥品漲幅控制在10％以下，輝瑞、賽諾菲等目前為止已調漲藥價的製藥巨頭，漲幅不超過5％。美國輝瑞擬調漲旗下逾60種藥品價格，漲幅在0.5％到5％之間，像是治療類風濕關節炎的暢銷藥Xeljanz及癌症藥物Ibrance、Inlyta，調漲約5％。法國大藥廠賽諾菲發言人寇斯（Ashleigh Koss）表示，會上調多項疫苗價格，漲幅在5％以內，1月稍後還將發布更多漲價資訊。3 Axis表示，英國藥廠葛蘭素史克生產的帶狀?疹疫苗Shingrix，以及白喉、破傷風、百日咳疫苗Pediarix，分別調漲7％、8.6％。3 Axis的資料顯示，去年美國市場計有860多種藥品漲價，平均漲幅在5％左右，但從調漲幅度及受影響的藥品數量來看，自2015年迄今藥廠已大幅放緩漲價速度。川普政府雖力求兌現調降處方藥價格的政策支票，但面臨PhRMA等製藥業團體的訴訟挑戰而遇阻。總統當選人拜登也矢言壓低藥品成本，打算交由聯邦醫療保險（Medicare）與藥商議價，他力挺民主黨國會議員推動相關立法，然國會預算辦公室估計，2029年前製藥業恐因此損失超過3,000億美元。'

In [71]:
test=requests.get('https://www.chinatimes.com/realtimenews/20201208004567-260408?chdtv')
china2=BeautifulSoup(test.text, 'html.parser')

In [14]:
china=BeautifulSoup(china_time[0].text, 'html.parser')

In [58]:
#print(china2.prettify())

In [42]:
#標題
china.find('h1', attrs={'class':'article-title'}).text

'印度將正式批准牛津疫苗 接種前全國先預演'

In [43]:
http[0]

'https://www.chinatimes.com/realtimenews/20210102001960-260408'

In [16]:
#日期
china.find('meta', property="article:published_time")['content'][0:10]

'2021-01-02'

In [17]:
#記者
author=china.find('div',attrs={'class':'author'}).text
author=author.replace('\n','')
author=author.replace(' ','')
author

'中央社'

In [18]:
#類別
china.find('meta',attrs={'property':'article:section'})['content']

'國際'

In [19]:
content=china.find('div',attrs={'class':'article-body'}).text
k=content.rfind('。')
content=content[:k+1]

In [20]:
content=content.replace('\n','')

In [21]:
def count_word(x):
    chinese=[]
    english=[]
    number=[]
    en=''
    num=''
    for i in list(x):

        if i in string.ascii_letters:
            en+=i
        elif i.isdigit():
            num+=i
        else:
            if en !='':
                english.append(en)
            if num!='':
                number.append(num)
            en=''
            num=''
            if i !=' ':
                chinese.append(i)
    words=len(english)+len(chinese)+len(number)
    return words

In [22]:
count_word(content)

1014

In [69]:
#圖片數
n_p=plen(china2.find_all('div',attrs={'class':'photo-container'}))
#影片數
v_p=len(china2.find_all('div',attrs={'class':'article-body','class':'video-container'}))

1

In [75]:
len(china2.find_all('div',attrs={'class':'article-body','class':'video-container'}))

2

In [73]:
china2.find_all('div',attrs={'class':"video-container"})

[<div class="video-container">
 <iframe allow="autoplay; encrypted-media" allowfullscreen="" height="315" src="https://www.youtube.com/embed/qWW1DZ8eUoo" width="560"></iframe>
 </div>,
 <div class="video-container">
 <iframe allow="autoplay; encrypted-media" allowfullscreen="" height="315" src="https://www.youtube.com/embed/-c4Xd7hZD1o" width="560"></iframe>
 </div>]

In [74]:
len(china.find_all('div',attrs={'class':'photo-container'}))

1

In [23]:
content

'印度武漢肺炎疫苗接種進入倒數階段，英國牛津大學與阿斯特捷利康（AstraZeneca）藥廠開發的疫苗昨天獲印度中央藥物標準控制組織批准，預料很快獲藥品管制總局放行。為了讓2019冠狀病毒疾病（COVID-19，武漢肺炎）配送與接種更加順暢，印度繼日前在4省進行疫苗配送與接種預演後，今天起在全國所有省市預演，以找出可能的問題。印度斯坦時報（Hindustan Times）今天報導，印度中央藥物標準控制組織（Central Drugs Standard Control Organisation，CDSCO）10人專家委員會昨天決議，建議授予緊急使用許可給牛津大學與阿斯特捷利康藥廠開發、印度血清研究所（Serum Institute of India）製造的疫苗。這款疫苗在全球稱為AZD1222，但由印度血清研究所製造且打上血清研究所標誌的疫苗稱為Covishield，。這款疫苗在印度進行臨床實驗時，曾有嚴重損傷神經的副作用案例發生。CDSCO專家委員會在建議授予Covishield緊急使用許可時，提出附帶條件，要求印度血清研究所應密切監測接種疫苗產生的副作用，並在當局正式批准緊急使用這款疫苗的頭2個月，每15天提交報告給當局。知情人士透露，印度藥品管制總局（Drugs Controller General of India）正在評估CDSCO專家委員會授予Covishield緊急使用許可的建議報告。外界預料，由於這款疫苗日前已獲得英國批准，印度藥品管制總局可能很快就會放行。印度血清研究所日前表示，已準備好5000萬劑Covishield疫苗，1月可先提供政府1000萬劑疫苗。美國輝瑞藥廠（Pfizer）與德國生技公司BioNTech 開發的武漢肺炎疫苗也申請緊急使用許可，CDSCO專家委員會正在審查。這款疫苗已在去年12月獲多國批准緊急使用，瑞輝藥廠希望在印度巴拉特生物科技國際公司（Bharat Biotech）開發的本土疫苗獲得批准前，有更多時間提供11月中旬起在印度進行第3階段臨床試驗的數據給專家委員會。印度日前挑選4省就武漢肺炎疫苗配送和接種流程進行預演。印度快報（Indian Express）引述消息人士透露，預演中發現一些村莊的代碼沒有成功對應到用於疫苗接種管理、追蹤和登記的Co-WIN行動裝置軟體（app）資料庫，疫苗接種小組人員必須手動操作，而這些村莊大